# Decoy Effect Model - Version 0.1

March 11 2023


In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mplc

from functions import Params, animate
from model import Decoy
from agents import Cell, MV, Drug

## OPTION 1: set parameters manually

In [ ]:
params = Params(0)

# Simulation Parameters
params.set_parameter("width", 100)                         # Width of array grid
params.set_parameter("height", 100)                        # Height of array grid
params.set_parameter("initial_population_0", 100)          # Initial population of SNVs
params.set_parameter("initial_population_1", 50)           # Initial population of SHVs
params.set_parameter("initial_population_2", 10)           # Initial population of RNVs
params.set_parameter("number_of_steps", 100)               # Time length of simulation (in ticks)
        
# Dossage controls
params.set_parameter("drug_dossage", 1000)                  # Drug dossage per site  
params.set_parameter("drug_dossage_time", 1000)            # Time period for new dossage (+/- 1%)

# Cell dynamics 
params.set_parameter("life_span", 100)                     # Age of cells natural death (+/- 10%)
params.set_parameter("division_age", 5)                    # Base age of cell division (+/- 5%)
params.set_parameter("fitness_cost_0", 1)                  # Fitness factor for SNVs (multiply division age)
params.set_parameter("fitness_cost_1", 1.5)                # Fitness factor for SHVs (multiply division age)
params.set_parameter("fitness_cost_2", 2)                  # Fitness factor for RNVs (multiply division age)
params.set_parameter("replacement_prob", 0.1)              # Probability of cell replacement  
params.set_parameter("kill_cell", 5)                       # Killing threshold of cell by the drug
        
# Vesicle dynamics
params.set_parameter("mv_prod_0", 0)                       # Background vesicule production per cell per tick
params.set_parameter("mv_prod_drug", 1)                    # Vesicule production per cell per tick per unit drug
params.set_parameter("mv_prod_max", 10)                    # Maximun vesicle production per cell per tick
params.set_parameter("kill_mv", 0.1)                       # Max drug per MV (MV is `killed` when saturated)
params.set_parameter("diff_mv", 0.1)                       # Rate of MV difussion (per tick per site)
params.set_parameter("mv_max", 50)                         # Maximun MVs in grid site (MV saturation)

# Drug dynamics
params.set_parameter("drug_abs_cell", 0.5)                 # Rate of drug absorption per cell per tick
params.set_parameter("drug_abs_mv", 0.1)                   # Rate of drug absorption per vesicle per tick
params.set_parameter("drug_decay", 0.01)                    # Rate of drug decay per tick    
params.set_parameter("diff_drug", 0.25)                    # Rate of drug difussion (per tick per site)    
params.set_parameter('drug_max', 1000)                       # Maximune drug in grid site (drug saturation)

## OPTION 2: read parameters from file

In [ ]:
params = Params(0, 'parameters_1.csv')

## Run single simulation

In [ ]:
simid = "sim_" + "{:04d}".format(params.id) 
decoy_model = Decoy(params, verbose = True)
outfile = "simulations/" + simid + ".npz"
cell_grid, mv_grid, drug_grid, time_series, cell_table = decoy_model.run_model(outfile)

## Run a batch, using parameter file

In [ ]:
params_tbl = pd.read_csv('parameters.csv')
params = Params()

N = len(params_tbl.index)
for i in range(N):
    
    params.read_parameters_from_tbl(params_tbl , i)
    simid = "sim_" + "{:04d}".format(params.id)
    outfile = "simulations/" + simid + ".npz"
    
    print(">>> Running Simulation: " + simid +" ... "+ str(i+1) + "/" + str(N), end="\r")
    
    decoy_model = Decoy(params, verbose = False)
    cell_grid, mv_grid, drug_grid, time_series, cell_table = decoy_model.run_model(outfile)
    
    

## Retreive data saved in previous run

In [ ]:
params_tbl = pd.read_csv('parameters.csv')

i = 0    # index of result of interest
params.read_parameters_from_tbl(params_tbl , i)
simid = "sim_" + "{:04d}".format(params.id) 
outfile = "simulations/" + simid + ".npz"

aux = np.load(outfile)
cell_grid = aux['cell_grid'] 
mv_grid = aux['mv_grid'] 
drug_grid = aux['drug_grid'] 
t_series = pd.DataFrame(aux['t_series'], columns = ['Step','SNVs','SHVs', 'RNVs'])
cell_table = pd.DataFrame(aux['cell_table'], columns = ['time','cell_id','phenotype', 
                                                        'age', 'x', 'y', 'drug'])


## Simple visualization

In [ ]:
# We need `notebook` for the anim to render in the notebook
%matplotlib notebook
N = len(t_series.index)

In [ ]:
cell_bins = [-1, 0, 1, 2, 3]
cell_colors = ("#000000", "#2b83ba", "#1a9641", "#d7191c")

cmap = mplc.ListedColormap(cell_colors)
norm = mplc.BoundaryNorm(cell_bins, len(cell_bins))

#plt.imshow(cells[-1, :, :], interpolation='nearest', cmap=cmap, norm=norm)
#plt.show()

anim = animate(cell_grid, N, cmap, norm,  "cells_"+ simid + ".mp4")

In [ ]:
mv_bins = np.linspace(0, np.max(mv_grid), 11, endpoint=True)
#mv_bins = np.linspace(0, MAX_MVS_SITE, 11, endpoint=True)
mv_colors = ("#ffffff", "#f7fcfd", "#e5f5f9", "#ccece6", "#99d8c9",
             "#66c2a4", "#41ae76", "#238b45", "#006d2c", "#00441b", "#000000")

cmap = mplc.ListedColormap(mv_colors)
norm = mplc.BoundaryNorm(mv_bins, len(mv_bins))

#plt.imshow(mv_grid[-1, :, :], interpolation='nearest', cmap=cmap, norm=norm)
#plt.show()

anim = animate(mv_grid, N, cmap, norm,  "mvs_"+ simid + ".mp4")

In [ ]:
drug_bins = np.linspace(0, np.max(drug_grid), 11, endpoint=True)
#drug_bins = np.linspace(0, MAX_DRUG_SITE, 11, endpoint=True)
drug_colors = ("#ffffff", "#ffffcc", "#ffeda0", "#fed976", "#feb24c", 
               "#fd8d3c", "#fc4e2a", "#e31a1c", "#bd0026", "#800026", "#000000")

cmap = mplc.ListedColormap(drug_colors)
norm = mplc.BoundaryNorm(drug_bins, len(drug_bins))

#plt.imshow(drug_grid[-1, :, :], interpolation='nearest', cmap=cmap, norm=norm)
#plt.show()
anim = animate(drug_grid, N, cmap, norm,  "drugs_"+ simid + ".mp4")